In [21]:
from bs4 import BeautifulSoup
from newspaper import Article
import requests
import pandas as pd

headers = requests.utils.default_headers()
headers.update({ 'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0'})
url = "https://www.covid19-archive.com/all-articles-on-one-page/"
req = requests.get(url, headers)
soup = BeautifulSoup(req.content, 'html.parser')
gg = soup.find_all("a")
c = 0
list_of_urls = []
for g in gg:
    string_link = g.get("href")
    if "https://web.archive.org/web/" in string_link:
        list_of_urls.append(string_link)
        c += 1
print("Total Count of Articles: ", c)

Total Count of Articles:  1757


In [30]:
d = {'Author': [], 'Date': [], "Text": []}
lost = 0
for art in list_of_urls:
    try:
        article = Article(art)
        article.download()
        article.parse()
        d['Author'].append(', '.join(article.authors))
        d['Date'].append(str(article.publish_date))
        d['Text'].append(article.text)
    except:
        lost += 1
    
raw_text = pd.DataFrame(data=d)
raw_text.to_csv("covid19rawDataset.csv")
print("Number of Failed Links: ", lost)

Building prefix dict from /usr/local/lib/python3.7/site-packages/jieba/dict.txt ...
Dumping model to file cache /var/folders/g0/hnbkmd414h51nyqlfy97813r0000gn/T/jieba.cache
Loading model cost 1.3275260925292969 seconds.
Prefix dict has been built succesfully.


Number of Failed Links:  8


In [32]:
raw_text[raw_text.Text != '']
raw_text.to_csv("covid19rawDataset.csv")

In [4]:
import pandas as pd
data = pd.read_csv("covid19rawDataset.csv")

import re
alphabets= "([A-Za-z])"
prefixes = "(Mr|St|Mrs|Ms|Dr)[.]"
suffixes = "(Inc|Ltd|Jr|Sr|Co)"
starters = "(Mr|Mrs|Ms|Dr|He\s|She\s|It\s|They\s|Their\s|Our\s|We\s|But\s|However\s|That\s|This\s|Wherever)"
acronyms = "([A-Z][.][A-Z][.](?:[A-Z][.])?)"
websites = "[.](com|net|org|io|gov)"
digits = "([0-9])"

def split_into_sentences(text):
    text = " " + text + "  "
    text = text.replace("\n"," ")
    text = re.sub(prefixes,"\\1<prd>",text)
    text = re.sub(websites,"<prd>\\1",text)
    if "Ph.D" in text: text = text.replace("Ph.D.","Ph<prd>D<prd>")
        
    text = re.sub("\s" + alphabets + "[.] "," \\1<prd> ",text)
    text = re.sub(acronyms+" "+starters,"\\1<stop> \\2",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>\\3<prd>",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>",text)
    text = re.sub(" "+suffixes+"[.] "+starters," \\1<stop> \\2",text)
    text = re.sub(" "+suffixes+"[.]"," \\1<prd>",text)
    text = re.sub(" " + alphabets + "[.]"," \\1<prd>",text)
    text = re.sub(digits + "[.]" + digits,"\\1<prd>\\2",text)
    
    if "”" in text: text = text.replace(".”","”.")
    if "\"" in text: text = text.replace(".\"","\".")
    if "!" in text: text = text.replace("!\"","\"!")
    if "?" in text: text = text.replace("?\"","\"?")
        
    text = text.replace(".",".<stop>")
    text = text.replace("?","?<stop>")
    text = text.replace("!","!<stop>")
    text = text.replace("<prd>",".")
    
    sentences = text.split("<stop>")
    sentences = sentences[:-1]
    sentences = [s.strip() for s in sentences]
    
    return sentences

In [7]:
with open("into_relex.txt", "a+") as file_object:
    file_object.write("text\n")
    for _,row in data.iterrows():
        text = split_into_sentences(str(row["Text"]))
        one_article = ' '.join(text)
        file_object.write(one_article + "\n")    